In [15]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import csv 
from datetime import datetime

from tensorflow import keras
from keras import layers
from keras import models
from keras import callbacks

from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout 
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, History 

from data_tools import load_preprocessed, dataPrep, nameModel

In [16]:
simPrefix = 'C:/Users/MOliv/Documents/PHY460/simFiles'
x, y = load_preprocessed(simPrefix, 'train')

Percentage of events with a NaN: 2.68


In [17]:
name = 'AvgPoolBatchNorm' 
i = 0
while (os.path.exists('AvgPoolBatchNorm/{}.h5'.format(name+str(i)))):
    i = i + 1 
name = name+str(i) 
numepochs = 100
prep = {'q':None, 't':False, 'normed':True, 'reco':'plane', 'cosz':False}

In [18]:
charge_input=keras.Input(shape=(10,10,2,))

conv1_layer = layers.Conv2D(64,kernel_size=3,padding='same',activation='relu')(charge_input)
batch1_layer = layers.BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,center=True,scale=True,beta_initializer="zeros",gamma_initializer="ones",moving_mean_initializer="zeros",moving_variance_initializer="ones",beta_regularizer=None,gamma_regularizer=None,beta_constraint=None,gamma_constraint=None)(conv1_layer)
conv2_layer = layers.Conv2D(32,kernel_size=3,padding='same',activation='relu')(batch1_layer)
batch2_layer = layers.BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,center=True,scale=True,beta_initializer="zeros",gamma_initializer="ones",moving_mean_initializer="zeros",moving_variance_initializer="ones",beta_regularizer=None,gamma_regularizer=None,beta_constraint=None,gamma_constraint=None)(conv2_layer)
conv3_layer = layers.Conv2D(16, kernel_size=3, padding='same',activation="relu")(batch2_layer) 
Apool_layer = layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding="valid", data_format=None) (conv3_layer)

flat_layer = layers.Flatten()(Apool_layer)
zenith_input=keras.Input(shape=(1,))
concat_layer = layers.Concatenate()([flat_layer,zenith_input])

dense1_layer = layers.Dense(256,activation='relu')(concat_layer)
dense2_layer = layers.Dense(256,activation='relu')(dense1_layer)
dense3_layer = layers.Dense(256,activation="relu")(dense2_layer)

output = layers.Dense(1)(dense3_layer)

model = models.Model(inputs=[charge_input,zenith_input],outputs=output,name=name)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])

In [19]:
model.summary()

Model: "MaxPoolBatchNorm0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 10, 10, 2)]  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 10, 10, 64)   1216        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 10, 10, 64)   256         conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 10, 10, 32)   18464       batch_normalization_2[0][0]      
__________________________________________________________________________________

In [20]:
len(np.sum(x,axis=(1,2,3)))

549773

In [21]:
th, _ = y['small_dir'].transpose()
len(th)

549773

In [22]:
x_i = dataPrep(x, y, **prep)

In [23]:
energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)
temp_y = energy

In [24]:
np.count_nonzero(x_i[1]==None)

0

In [ ]:
csv_logger = CSVLogger('models/{}'.format(name))
early_stop = EarlyStopping(monitor="val_loss",min_delta=0,patience=10,verbose=0,mode="auto",baseline=None,restore_best_weights=False,) 
callbacks = [early_stop, csv_logger]
history = model.fit(
    {"input_1":x_i[0],"input_2":x_i[1].reshape(-1,1)}, temp_y, epochs=numepochs,validation_split=0.15,callbacks=callbacks)

Epoch 1/100
14354/14354 [==============================] - 313s 22ms/step - loss: 0.0651 - mae: 0.1879 - mse: 0.0651 - val_loss: 0.0371 - val_mae: 0.1452 - val_mse: 0.0371 - ETA: 3s - loss: 0.0652 - mae: 0.1881 - ETA: 2s 
Epoch 2/100
14354/14354 [==============================] - 266s 19ms/step - loss: 0.0391 - mae: 0.1514 - mse: 0.0391 - val_loss: 0.0318 - val_mae: 0.1393 - val_mse: 0.0318
Epoch 3/100
14354/14354 [==============================] - 247s 17ms/step - loss: 0.0349 - mae: 0.1422 - mse: 0.0349 - val_loss: 0.0269 - val_mae: 0.1228 - val_mse: 0.0269
Epoch 4/100
14354/14354 [==============================] - 260s 18ms/step - loss: 0.0299 - mae: 0.1299 - mse: 0.0299 - val_loss: 0.0243 - val_mae: 0.1183 - val_mse: 0.0243
Epoch 5/100
 1899/14354 [==>...........................] - ETA: 3:14 - loss: 0.0287 - mae: 0.1262 - mse: 0.0287- ETA: 3:14 - loss: 0.0286 - mae: 0.1262 - mse: 0.0

In [ ]:
np.save("AvgPoolBatchNorm.npy",prep)
#make a folder and make the file save to that folder 
model.save('models/%s.h5' % name)
f = open("results.txt", "a")
now = datetime.now()
f.write("{}\t{}\tepochs:{}\tloss:{},{}\n".format(
    now.strftime("%m/%d/%Y %H:%M:%S"),
    name,
    len(history.history['loss']),
    history.history['loss'][len(history.history['loss'])-1],
    history.history['val_loss'][len(history.history['loss'])-1]
))
f.close()